In [1]:
import numpy as np
import torch
import local_utils
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from local_utils import ResidualBlock
from torch import nn

### W tej części ćwiczenia, wczytamy nauczony model zmiennoprzecinkowy MiniResNet, skwantyzujemy go do postaci stałoprzecinkowej i na koniec skompilujemy go.

# 1. Dane ewaluacyjne

Zaczynamy od ponownego stworzenia generatora danych na bazie danych MNIST:

Wystarczy nam sama część testowa. Ustawiamy `batch_size` na 1.

In [2]:
test_dataset = ... #TODO
test_loader = ... #TODO

Dodatkowo przygotujemy plik w formacie `.npz` na podstawie danych testowych. Wykorzystamy go do ewaluacji modelu na docelowej platformie Kria.

Uzupełnij wektory `quantization_data` oraz `quantization_labels` danymi z `test_loadera`. Wykorzystaj do tego pętle `for` oraz `.append` (Przykład wykorzystania DataLoader'a z pętlą `for` przedstawiono w 1 części podczas wczytywania danych).

Następnie każdy wektor z osobna połącz funkcją `torch.cat` z parametrem `dim=0` i przekonwertuj je do formatu `ndarray` za pomocą `.numpy()`. 

Zapisz je funkcją np.savez.

In [3]:
quantization_data = []
quantization_labels = []

#TODO
#Fill quantization vectors

train_X = ... #TODO
train_Y = ... #TODO

np.savez('eval_MNIST.npz', data=..., targets=...) #TODO

print('Done')

Done


# 2. Inicjalizacja modelu zmiennoprzecinkowego

Tworzymy taką samą klasę sieci MiniResNet jak w pierwszej części ćwiczenia.

In [4]:
class MiniResNet(nn.Module):
    #TODO

Tworzymy model i wgrywamy wagi z pliku MNIST.pth. Zapisujemy go do urządzenia (w dockerze dostępny jest tylko CPU!) i ustawiamy go na ewaluację `.eval()`.

In [5]:
device = ... #TODO

net = ... #TODO
pretrainedModel = ... #TODO
net.load_state_dict(pretrainedModel['model'])
net.to(device)
#TODO change to eval

print(device)

cpu


# 3. Ewaluacja modelu zmiennoprzecinkowego

Przed przystąpieniem do kwantyzacji, wykonamy szybką ewaluację modelu zmiennoprzecinkowego. Sprawdzimy, czy dane są dobrze przygotowane i czy model został odpowiednio zapisany i wczytany. Wczytujemy metrykę Accuracy z `local_utils`.

In [6]:
def evaluate(model,
             dataloader,
             evaluator
             ):
    tm = local_utils.TimeMeasurement("Evaluation", len(dataloader))
    with torch.no_grad(), tm:
        score = 0.0
        cntr = 0
        for i, XY in enumerate(dataloader):
            X = XY[0]
            Y = XY[1:]
            y_pred = model(X)
            score = score*cntr + X.shape[0]*evaluator(y_pred, *Y)
            cntr += X.shape[0]
            score /= cntr
            print("\rEvaluation {}/{}. Score = {}".format(i,len(dataloader), score),end='')
        
        print("\rEvaluation {}/{}. Score = {}".format(len(dataloader),len(dataloader), score),end='\n')
    print(tm)

In [7]:
metric = ... #TODO

# You can evaluate your floating point model first 
evaluate(net, test_loader, metric)

Evaluation 10000/10000. Score = 0.9822999835014343
Execution time: 0.0:0.0:29:111, processed 10000 frames, throughput: 343.51067327585497 fps.


#### Jeżeli wszystko działa poprawnie, a uzyskana dokładność jest na podobnym poziome jak podczas uczenia, możemy przejść do kwantyzacji.

# 4. Kwantyzacja modelu zmiennoprzecinkowego

### Vitis AI Quantizer dla Post Training Quantization składa się z dwóch części.
Pierwszą częścią jest kalibracja (mode='calib') - Vitis AI Quantizer analizuje model i dostosowuje parametry kwantyzacji.
 
Drugą częścią jest ewaluacja/testowanie (mode='test') - sprawdzana jest dokładność modelu (nie powinna być duża zmiana) i model jest eksportowany do formatu .xmodel.

### Do obu części wykorzystamy funkcję quantize.

Funkcja wykorzystuje kwantyzator dla PyTorch z gita Vitis AI: https://github.com/Xilinx/Vitis-AI/tree/1.4/tools/Vitis-AI-Quantizer/vai_q_pytorch

In [8]:
def quantize(float_model:torch.nn.Module, 
             input_shape:tuple,
             quant_dir:str, 
             quant_mode:str, 
             device:torch.device,
             dataloader,
             evaluator):
    """
    :param float_model: float model with loaded weights
    :param input_shape: shape of input(CH,W,H)
    :param quant_dir: path to directory with quantized model components
    :param quant_mode: quant_mode in ['calib', 'test'] 
    :param data_loader: data_loader - for 'calib' must be batch_size == 1
    :param evaluator: fcn/obj like: fcn(y_pred, y_ref) -> float 
    """
    tm = local_utils.TimeMeasurement("Quantization", len(dataloader))
    with tm:
        # available in docker or after packaging 
        # vitis-AI-tools/..../pytorch../pytorch_nndct
        # and installing the package
        from pytorch_nndct.apis import torch_quantizer, dump_xmodel
        # model to device
        model = float_model.to(device)

        # Force to merge BN with CONV for better quantization accuracy
        optimize = 1

        rand_in = torch.randn(input_shape)
        print("get qunatizer start")
        try:
            quantizer = torch_quantizer(
                quant_mode, model, rand_in, output_dir=quant_dir, device=device)
        except Exception as e:
            print("exception:")
            print(e)
            return
        print("get qunatizer end")

        print("get quantized model start")
        quantized_model = quantizer.quant_model
        print("get quantized model end")

        # evaluate
        print("testing st")
        evaluate(quantized_model, dataloader, evaluator)
        print("testing end")

        # export config
        if quant_mode == 'calib':
            print("export config")
            quantizer.export_quant_config()
            print("export config end")
        # export model
        if quant_mode == 'test':
            print("export xmodel")
            quantizer.export_xmodel(deploy_check=False, output_dir=quant_dir)
            print("export xmodel end")
    print(tm)

Zaczynamy od kalibracji. Jako wejście funkcji podajemy:
- float_model - model zmiennoprzecinkowy, który uzyskaliśmy podczas uczenia,
- input_shape - wymiar danych wejściowych w formacie [batch, CH, W, H],
- quant_dir - folder do którego zostanie zapisany wynik kwantyzacji,
- quant_mode - do wyboru 'calib' albo 'test',
- device - urządzenie na którym zostaną wykonane obliczenia (CPU),
- dataloader - dane na którym będą wykonane obliczenia,
- evaluator - metryka, względem której będzie sprawdzana dokładność

### Uwaga! Kwantyzacja w procesie kalibracji jest wolna. W przypadku dużych modeli i dużych wymiarów danych, nie można przesadzić z ilością danych.

In [9]:
# Quantize model - calib - is slow

#TODO
quantize(float_model=..., 
         input_shape=...,
         quant_dir='quant_dir',
         quant_mode='calib',
         device=...,
         dataloader=...,
         evaluator=...
         )

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

[VAIQ_NOTE]: Loading NNDCT kernels...
get qunatizer start

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing MiniResNet...

[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir/MiniResNet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
testing st
Evaluation 10000/10000. Score = 0.9817000031471252
Execution time: 5.0:0.0:30:416, processed 10000 frames, throughput: 30.264823388501018 fps.
testing end
export config

[VAIQ_NOTE]: =>Exporting quant config.(quant_dir/quant_info.json)
export config end
Execution time: 5.0:0.0:31:397, processed 10000 frames, throughput: 30.175226697334523 fps.


Po udanej kalibracji, czas na testowanie i zapisanie modelu. Uruchamiamy funkcję ze zmienionym parametrem mode na 'test'.

Proces ten jest szybszy od kalibracji.

In [10]:
# Quantize model - test - is faster

#TODO
quantize(float_model=..., 
         input_shape=...,
         quant_dir='quant_dir', # directory for quantizer results
         quant_mode='test',
         device=...,
         dataloader=...,
         evaluator=...
         )

get qunatizer start

[VAIQ_NOTE]: Quantization test process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing MiniResNet...

[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir/MiniResNet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
testing st
Evaluation 10000/10000. Score = 0.9824000000953674
Execution time: 0.0:0.0:57:169, processed 10000 frames, throughput: 174.9184548731919 fps.
testing end
export xmodel

[VAIQ_NOTE]: =>Converting to xmodel ...

[VAIQ_NOTE]: =>Successfully convert 'MiniResNet' to xmodel.(quant_dir/MiniResNet_int.xmodel)
export xmodel end
Execution time: 0.0:0.0:57:426, processed 10000 frames, throughput: 174.1343355669612 fps.


Po testowaniu, należy skompilować model. Podajemy odpowiednio parametry:

- --xmodel - ścieżka do zapisanego modelu (zapisany jest w folderze podanym podczas kwantyzacji jako parametr 'quant_dir'
- --arch - podajemy plik arch.json, który znajdował się w pliku. Jest to numer (fingerprint), który określa typ DPU sprzętu docelowego
- --net_name - nazwa naszego modelu po kompilacji - dowolna
- --output_dir - folder do którego zapisany zostanie model

In [11]:
# compile model
!vai_c_xir --xmodel 'quant_dir/MiniResNet_int.xmodel' --arch arch.json --net_name MiniResNet_qu --output_dir build

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B4096_MAX_BG2
[UNILOG][INFO] Graph name: MiniResNet, with op num: 130
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/build/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/build/MiniResNet_qu.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is 6f353093d80acd35dd41b979bfe4e8e1, and has been saved to "/workspace/build/md5sum.txt"


Teraz przejdziemy do testowania modelu na sprzęcie docelowym.

#### Wersja 1: Podłączenie do sieci
Zanim podłączysz zasilanie do Kria, podepnij kabel USB do komputera, a kabel Ethernetowy do sieci, w której znajduje się komputer.

Podłącz Kria do zasilania i poczekaj minutę, aż system się uruchomi.

Uruchom `cutecom` z `sudo`. Włącz port odpowiadający do Kria. Jeżeli pojawi się napis `kria login:` zaloguj się:

`login: ubuntu`

`hasło: Xilinx123`

Po zalogowaniu się, powinny pojawić się informacje systemowe. Nas interesuje adres `IPv4` dla `eth0`. Skopiuj go i dodaj do niego `:9090` - przykładowa wartość `192.168.1.26:9090`. Wklej to w przeglądarce. Powinien pojawić się kolejny Jupyter. Zaloguj się do niego hasłem:

`xilinx`

#### Wersja 2: Podłączenie do komputera
Zanim podłączysz zasilanie do Kria, podepnij kabel USB do komputera oraz kabel Ethernetowy pomiędzy Kria a PC. Na PC włącz ustawienia sieci `Wired Setting` -> `IPv4` -> `Shared to other computers`. Włącz zasilanie płytki.
Uruchom `cutecom` z `sudo`. Włącz port odpowiadający do Kria. Jeżeli pojawi się napis `kria login:` zaloguj się:

`login: ubuntu`

`hasło: Xilinx123`

Po zalogowaniu się, powinny pojawić się informacje systemowe. Nas interesuje adres `inet` dla `eth0`. Skopiuj go i dodaj do niego `:9090` - przykładowa wartość `10.42.0.47:9090`. Wklej to w przeglądarce. Powinien pojawić się kolejny Jupyter. Zaloguj się do niego hasłem:

`xilinx`

#### Przesyłanie plików
Stwórz nowy folder i nazwij go `PSRA_Lab`. Przenieś do niego odpowiednio pliki:
- dpu.bit, 
- dpu.hwh, 
- dpu.xclbin, 
- eval_MNIST.npz lub tak jak nazwałeś swój plik z danymi do ewaluacji
- MiniResNet_compiled.xmodel lub tak jak nazwałeś swój skompilowany plik

Można to wykonać komendą `scp`, ale łatwiej jest przeciągnąć pliki z folderu do Jupyter Notebook'a.

# UWAGA! 

Jeżeli wystąpi problem z adresami IPv4 Kria (po podłaczeniu kilku płytek do jednej sieci, każda z nich będzie miała taki sam adres), należy sprawdzić, czy działa komenda w konsoli `cutecom`:

`ifconfig`

Jeżeli nie, to należy zainstalować poprzez:

 `sudo apt install net-tools`.  
 
Po tym należy odpiąć kabel Ethernet z Kria, poczekać kilka sekund i wpisać w konsole `cutecom`:

`hostname -I`

Jeżeli konsola nie zwróci żadnego błędu oraz żadnego aresu IP to wpisz w konsole `cutecom`:

`sudo ifconfig eth0 192.168.1.x netmask 255.255.255.0`

Tutaj podany adres powinien być taki sam jak przykładowa wartość wyżej. Ustawiamy wartość `x` na inną niż była np. 123. Chcemy uniknąć konfliktu pomiędzy płytkami ale również komputerami. Po tym znowu ponawiamy:

`hostname -I`.

Powinien pojawić sie ustawiony przez nas adres. Podpinamy kabel Ethernet i uruchamiamy w przeglądarce Jupyter z ustawionym adresem IP.